In [ ]:
### Install and import libraries
!pip install pandas openpyxl
!pip install dataset
!pip install unsloth
!pip install gradio
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install sentence-transformers bert-score nltk rouge-score
import pandas as pd
import json
from google.colab import files
import numpy as np
from datasets import Dataset
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
### Downloading of model

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Preprocessing of Multiple-Choice Questions

In [ ]:
"""
Script for processing multiple-choice questions from an Excel file, preparing data for model training.
The script performs the following:
1. Reads multiple-choice questions and metadata from an Excel file.
2. Formats the data with options, ranks, and justifications for each question.
3. Converts the data into a JSON format and saves it to a file.
4. Prepares a Hugging Face dataset for training, including tokenization and formatting.

Modules:
    pandas: For reading and processing Excel data.
    numpy: For handling numerical operations.
    json: For saving and loading JSON files.
    datasets: For creating and managing Hugging Face datasets.
    transformers: For tokenization and formatting.

Functions:
    preprocess_function_mc: Prepares the dataset by tokenizing questions and answers and aligning labels.

Usage:
    - Ensure the Excel file "Teliko.xls" exists in the working directory with the correct sheet and columns.
    - Replace `tokenizer` with the appropriate tokenizer for your language model.
    - Customize parameters like max_length, padding, and truncation based on the specific model being used.
"""

import pandas as pd
import numpy as np
import json
from datasets import Dataset

# Step 1: Load the Excel file
df = pd.read_excel("Teliko.xls", sheet_name='multiple choice questions')
"""
Reads the 'multiple choice questions' sheet from the Excel file into a pandas DataFrame.
"""

# Define column names for the final DataFrame
columns = [
    'ID', 'question', 'Choice A', 'metadata A.rank', 'metadata A.justification',
    'Choice B', 'metadata B.rank', 'metadata B.justification',
    'Choice C', 'metadata C.rank', 'metadata C.justification',
    'Choice D', 'metadata D.rank', 'metadata D.justification'
]

# Step 2: Initialize processing structures
multiple_choices = []
options = [["None", "None", "None", "None"]]
justifications = [["None", "None", "None", "None"]]
ranks = [["None", "None", "None", "None"]]
questions = [df.columns[1].replace('\n', '')]
id = 0
current_option_id = 0

# Process the DataFrame row by row
for i in range(len(df)):
    if df.iloc[i, 1] in ["text", "text "]:  # Extract options
        options[id][current_option_id] = str(df.iloc[i, 2]).replace('\n', '')
    elif df.iloc[i, 1] == 'justification':  # Extract justifications
        justifications[id][current_option_id] = str(df.iloc[i, 2]).replace('\n', '')
    elif df.iloc[i, 1] in ['rank', 'rank ']:  # Extract ranks
        ranks[id][current_option_id] = df.iloc[i, 2]
        current_option_id += 1
    elif df.iloc[i, 0] == 'question':  # Process new question
        current_option_id = 0
        id += 1
        options.append(["None", "None", "None", "None"])
        justifications.append(["None", "None", "None", "None"])
        ranks.append(["None", "None", "None", "None"])
        questions.append(df.iloc[i, 1].replace('\n', ''))

# Combine processed data into records
for i in range(len(questions)):
    multiple_choices.append([
        i, questions[i], options[i][0], ranks[i][0], justifications[i][0],
        options[i][1], ranks[i][1], justifications[i][1],
        options[i][2], ranks[i][2], justifications[i][2],
        options[i][3], ranks[i][3], justifications[i][3]
    ])

# Step 3: Create a DataFrame and convert it to JSON
df = pd.DataFrame(multiple_choices, columns=columns)

df.to_excel('phishing_multiple_choice_questions_new_vfinal.xlsx')
df = pd.read_excel('./phishing_multiple_choice_questions_new_vfinal.xlsx')

dict_data = df.to_dict(orient='records')
choices = ['A', 'B', 'C', 'D']
for row in dict_data:
    options_num = 0
    row['question'] = row['question'].replace('\n', "")
    for choice in choices:
        if pd.notna(row[f'metadata {choice}.rank']) and row[f'metadata {choice}.rank'] != "None":
            answer_choice = row[f'Choice {choice}'].replace('\n', "")
            answer_rank = int(row[f'metadata {choice}.rank'])
            answer_justification = row[f'metadata {choice}.justification'].replace('\n', "")
            answer = f"{choice}) {answer_choice} - Justification: {answer_justification} - Rank: {answer_rank}"
            row[f'choice_{choice}'] = answer
            row[f'Choice {choice}'] = answer_choice
            options_num += 1
            if answer_rank == 1:
                row['answer'] = answer
        else:
            row[f'choice_{choice}'] = 'None'
            row[f'Choice {choice}'] = 'None'
    row['options_num'] = options_num

# Step 4: Save the processed data to JSON
json_data = json.dumps(dict_data, indent=4)
with open('output_with_nested_metadata.json', 'w') as json_file:
    json_file.write(json_data)

# Load JSON into a Hugging Face Dataset
with open('output_with_nested_metadata.json', 'r') as json_file:
    json_data = json.load(json_file)

def preprocess_function_mc(examples):
    """
    Tokenizes multiple-choice questions and aligns labels for model training.

    Args:
        examples (dict): A batch of examples containing questions, choices, and answers.

    Returns:
        dict: Tokenized input with aligned labels for Hugging Face datasets.

    Notes:
        - Tokenizes input questions and options using the provided tokenizer.
        - Aligns correct answers with the corresponding choice labels.
        - Pads and truncates inputs to fit the model's maximum input length.
        - Replaces padding tokens in labels with -100 for loss function masking.
    """
    inputs = []
    labels = []

    questions = [question for question in examples['question']]
    answers = [answer for answer in examples['answer']]
    choices = {
        'A': [choice for choice in examples['choice_A']],
        'B': [choice for choice in examples['choice_B']],
        'C': [choice for choice in examples['choice_C']],
        'D': [choice for choice in examples['choice_D']]
    }

    surplus = 0
    while surplus < len(questions):
        num = 0
        for option in choices:
            if examples[f'choice_{option}'][surplus] != "None":
                num += 1
                input_text = f"Question: {questions[surplus]}\nOption: {choices[f'{option}'][surplus]}\nAnswer:"
                inputs.append(input_text)
                labels.append(answers[surplus] if choices[f'{option}'][surplus] == answers[surplus] else "")
        surplus += num

    tokenized_inputs = tokenizer(
        inputs,
        padding='max_length',
        truncation=True,
        max_length=512,  # Adjust this based on model input limits
        return_tensors="pt",
    )

    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(
            labels,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors="pt"
        )

    labels_ids = tokenized_labels['input_ids']
    # Replace padding token IDs in labels with -100 so they are ignored by the loss
    labels_ids[labels_ids == tokenizer.pad_token_id] = -100
    tokenized_inputs['labels'] = labels_ids
    return tokenized_inputs

dataset = Dataset.from_dict({
    "question": [item["question"] for item in json_data],
    "choice_A": [item["choice_A"] for item in json_data],
    "choice_B": [item["choice_B"] for item in json_data],
    "choice_C": [item["choice_C"] for item in json_data],
    "choice_D": [item["choice_D"] for item in json_data],
    "answer":   [item["answer"] for item in json_data],
    "options_num": [item["options_num"] for item in json_data],
    "Choice A": [item["Choice A"] for item in json_data],
    "Choice B": [item["Choice B"] for item in json_data],
    "Choice C": [item["Choice A"] for item in json_data],
    "Choice D": [item["Choice D"] for item in json_data],
})

# Split dataset into training and test sets
split_dataset = dataset.train_test_split(test_size=0.3,seed=0)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

train_dataset = Dataset.from_dict({
    "question": [item["question"] for item in train_dataset for i in range(item['options_num'])],
    'choice_A': [item['choice_A'] for item in train_dataset for i in range(item['options_num'])],
    'choice_B': [item['choice_B'] for item in train_dataset for i in range(item['options_num'])],
    'choice_C': [item['choice_C'] for item in train_dataset for i in range(item['options_num'])],
    'choice_D': [item['choice_D'] for item in train_dataset for i in range(item['options_num'])],
    'answer': [item['answer'] for item in train_dataset for i in range(item['options_num'])],
})

tokenized_train_dataset = train_dataset.map(preprocess_function_mc, batched=True)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['question', 'choice_A', 'choice_B', 'choice_C', 'choice_D'])
tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

#### All of dataset
dataset = Dataset.from_dict({
    "question": [item["question"] for item in dataset for i in range(item['options_num'])],
    'choice_A': [item['choice_A'] for item in dataset for i in range(item['options_num'])],
    'choice_B': [item['choice_B'] for item in dataset for i in range(item['options_num'])],
    'choice_C': [item['choice_C'] for item in dataset for i in range(item['options_num'])],
    'choice_D': [item['choice_D'] for item in dataset for i in range(item['options_num'])],
    'answer': [item['answer'] for item in dataset for i in range(item['options_num'])],
})

tokenized_dataset = dataset.map(preprocess_function_mc, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['question', 'choice_A', 'choice_B', 'choice_C', 'choice_D'])
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



Training of the model on the the entire dataset

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_dataset,
    #dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 100, # Set this for 1 full training run.
        # max_steps = 1000,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()
model.save_pretrained('./multiple_choice_questions_model')